<center>Universidade Estadual Paulista - UNESP</center>
<center>Faculdade de Engenharia de Ilha Solteira - FEIS</center>
<center>Departamento de Engenharia Mecânica - DEM</center>
<center>Programa de Pós-Graduação em Engenharia Mecânica - PPGEM</center>
<br>
<center><b>Disciplina: Dinâmica dos Sistemas Mecânicos</b></center>
<center><b>Pêndulo Simples parametrizado por x e y via Princípio de Hamilton</b></center>
<br>
<center><b>Docente:</b> Samuel da Silva</center>
<center><b>Discentes:</b> Estevão Fuzaro de Almeida</center>
<center>Henrique Cordeiro Novais</center>

---

### **1. Fazendo por $x$ e $y$.**

In [ ]:
# Vínculo holônomo
cnstr = x**2 + y**2 - l**2
cnstrx = smp.diff(cnstr,x)
cnstry = smp.sqrt(l**2 - x**2)
eq = - cnstrx/(2*cnstry)
eq

In [ ]:
# Principio de Hamilton
dLdx = smp.diff(L,x)
dLdxdot = smp.diff(L,xdot)
dLdy = smp.diff(L,y)
dLdydot = smp.diff(L,ydot)
Ham_x = dLdx - smp.diff(dLdxdot,t)
Ham_y = dLdy - smp.diff(dLdydot,t)
Ham = Ham_x + Ham_y*eq
Ham

In [ ]:
# Parâmetros do sistema
g = 9.81  # aceleração da gravidade
l = 1.0   # comprimento do pêndulo

# Intervalo de tempo para integração
t_start = 0.0
t_end = 20.0
t_step = 0.001
t_span = np.arange(t_start, t_end, t_step)

## HAMILTON --------------------------------------------------------
def pendulum_ham(z, t):
    z1 = z[0]
    z2 = z[1]
    dz1_dt = z2
    dz2_dt = -((g*(l**2 - z1**2)**(3/2) + l**2*(z2**2))*z1)/(l**2*z1**2 + (l**2 - z1**2)**2 - z1**4)
    return [dz1_dt, dz2_dt]

# Condições iniciais
z0 = [l*np.sin(np.pi/3), 0.0]  # z1(0), z2(0)

# Resolvendo o sistema de equações diferenciais
solution = odeint(pendulum_ham, z0, t_span)

# Obtendo as soluções para z1, z2, z3 e z4
x_sol = solution[:, 0]
y_sol = (l**2 - x_sol**2)

## TRADICIONAL --------------------------------------------------------
def pendulum_an(z, t):
    z1 = z[0]
    z2 = z[1]
    dz1_dt = z2
    dz2_dt = -g/l*np.sin(z1)
    return [dz1_dt, dz2_dt]

# Condições iniciais
z0 = [np.pi/3, 0.0]  # z1(0), z2(0)

# Resolvendo o sistema de equações diferenciais
solution_an = odeint(pendulum_an, z0, t_span)

# Obtendo as soluções para z1, z2
x_sol_an = l*np.sin(solution_an[:, 0])
y_sol_an = l*np.cos(solution_an[:, 0])

# Plotando as soluções
plt.figure(figsize=(10, 8))
plt.plot(x_sol, -y_sol, linestyle='-', color='red', linewidth=3, label=r'$\mathcal{H}$')
plt.plot(x_sol_an, -y_sol_an, linestyle=':', color='blue', linewidth=1, label=r'$\theta$')
plt.xlabel('$x$')
plt.ylabel('$y$')
plt.title('Pêndulo - Princípio de Hamilton')
plt.legend()
plt.grid(True)
plt.xlim(-1.0,1.0)
plt.ylim(-1.0,0.0)
plt.axis()
ax = plt.gca()
ax.set_aspect('equal', adjustable='box')
plt.show()